In [1]:
import sys
sys.path.append('/home/mts87985/ml-tomo/super-resolution-ml/')
import numpy as np

In [2]:
datapath='segmentation/data/train/'
valpath = 'segmentaiton/data/test/'
img_dir='noiseless/'
mask_dir='label/'
ftypes=['.tiff']

In [3]:
image_shape = (1280, 1280)
patch_shape = (64, 64)
patch_range = []

In [4]:
from data_handeling.generators import mask_patch_from_file

myGene = mask_patch_from_file(datapath,
                           img_dir, mask_dir,
                           patch_shape, image_shape,
                           types = ftypes, patch_range=patch_range,
                           debug=False,
                           batch_size = 1,
                           normalise_images=False)

valGene = mask_patch_from_file(valpath,
                           img_dir, mask_dir,
                           patch_shape, image_shape,
                           types = ftypes, patch_range=patch_range,
                           debug=False,
                           batch_size = 1,
                           normalise_images=False)

In [5]:
from models.u_net.model import unet
import models.losses.custom_loss_functions as losses
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy

model = unet(input_size = (patch_shape[0], patch_shape[1], 1))
opt = Adam()
#model.compile(loss=losses.weighted_cross_entropy(3), optimizer=opt)
model.compile(loss=binary_crossentropy , optimizer=opt, metrics=['accuracy'])

In [ ]:
epochs = 4
steps_per_epoch = 400
model.fit(myGene, steps_per_epoch=steps_per_epoch, epochs=epochs, validation_data=valGene, validation_steps=100)
model.save_weights('saved_weights.hdf5')

In [ ]:
model.load_weights('saved_weights.hdf5')

In [ ]:
model.save('saved_model.h5')

In [ ]:
from  utils.tools import inference_binary_segmentation
from tensorflow.keras.models import load_model

datapath = 'data/segmentation/test/noiseless/'
patch_shape = (64, 64)
image_shape = (1280, 1280)
savepath = './inferred_masks/'

inference_binary_segmentation(datapath, patch_shape, image_shape, model,
                 file_prefix='binary_mask', savepath=savepath, fig_size=(20, 8))

In [ ]:
import imageio
a = imageio.imread('data/test/label/00000.tiff')
import matplotlib.pyplot as plt
plt.imshow(a)

In [ ]:
np.min(a)

In [ ]:
a.shape